# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the data.
2. Build a feedforward neural network to classify traffic signs.
3. Build a convolutional neural network to classify traffic signs.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

## Load the Data

Start by importing the data from the pickle file.

In [1]:
import numpy as np

# TODO: Implement load the data here.
from zimpy.datasets.german_traffic_signs import GermanTrafficSignDataset

data = GermanTrafficSignDataset()
data.configure(one_hot=True, train_validate_split_percentage=0)

X_train = data.train_orig
y_train = data.train_labels

print(data)

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

 
One-Hot Encoded:             True
Train/Validation Split %:    0
 
Training size:               39209
Validation size:             0
Testing size:                12630
Predicting size:             13
Total classes:               43
 
Training orig shape:         (39209, 32, 32, 3)
Training gray shape:         (39209, 32, 32)
Training flat shape:         (39209, 1024)
 
Validation orig shape:       (0,)
Validation gray shape:       (0,)
Validation flat shape:       (0,)
 
Testing orig shape:          (12630, 32, 32, 3)
Testing gray shape:          (12630, 32, 32)
Testing flat shape:          (12630, 1024)
 
Predicting orig shape:       (13, 32, 32, 3)
Predicting gray shape:       (13, 32, 32)
Predicting flat shape:       (13, 1024)
 
Training label shape:        (39209, 43)
Validation flat label shape: (0,)
Testing gray label shape:    (12630, 43)
 
Class counts:
 
Sign names:
 
  class - no. train - sign name
 
  0 -  210 - Speed limit (20km/h)
  1 -  2220 - Speed limit (30km/h)
  2 

## Normalize the data

Now that you've loaded the training data, normalize the input so that it has a mean of 0 and a range between -0.5 and 0.5.

In [2]:
# TODO: Implement data normalization here.
X_train = data.normalize_data(X_train)

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(round(np.mean(X_train)) == 0), "The mean of the input data is: %f" % np.mean(X_train)
assert(np.min(X_train) == -0.5 and np.max(X_train) == 0.5), "The range of the input data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [3]:
from keras.layers import Dense, Activation, Flatten
from keras.models import Sequential
from keras.utils import np_utils

# TODO: Build a two-layer feedforward neural network with Keras here.
# as first layer in a sequential model:
model = Sequential(name='input')
model.add(Dense(128, input_dim=32*32*3, name='hidden1'))
# model.add(Flatten())
model.add(Activation('relu'))
model.add(Dense(data.num_classes))
model.add(Activation('softmax', name='output'))

model.summary()

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(model.get_layer(name="hidden1").input_shape == (None, 32*32*3)), "The input shape is: %s" % model.get_layer(name="hidden1").input_shape
assert(model.get_layer(name="output").output_shape == (None, 43)), "The output shape is: %s" % model.get_layer(name="output").output_shape

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
hidden1 (Dense)                  (None, 128)           393344      dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 128)           0           hidden1[0][0]                    
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 43)            5547        activation_1[0][0]               
____________________________________________________________________________________________________
output (Activation)              (None, 43)            0           dense_1[0][0]                    
Total params: 398891
______________________________________________________________________

## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [4]:
# TODO: Compile and train the model here.
batch_size = 128
nb_classes = data.num_classes
nb_epoch = 2

X_train = X_train.reshape(X_train.shape[0], 32*32*3)
X_train = X_train.astype('float32')

model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1)

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][0] > 0.5), "The training accuracy was: %.3f" % history.history['acc']

Epoch 1/2
39209/39209 [==============================] - 3s - loss: 1.6054 - acc: 0.5847     
Epoch 2/2
39209/39209 [==============================] - 3s - loss: 0.7299 - acc: 0.8125     


## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [6]:
from sklearn.model_selection import train_test_split

# TODO: Split some of the training data into a validation dataset.
X_train, X_val, y_train, y_val = train_test_split(
            X_train,
            y_train,
            test_size=0.25,
            random_state=832224)

# TODO: Compile and train the model to measure validation accuracy.
history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))

score = model.evaluate(X_val, y_val, verbose=1)

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(round(X_train.shape[0] / float(X_val.shape[0])) == 3), "The training set is %.3f times larger than the validation set." % X_train.shape[0] / float(X_val.shape[0])
assert(history.history['val_acc'][0] > 0.6), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 22054 samples, validate on 7352 samples
Epoch 1/2
22054/22054 [==============================] - 3s - loss: 0.4033 - acc: 0.8957 - val_loss: 0.3321 - val_acc: 0.9195
Epoch 2/2
7296/7352 [============================>.] - ETA: 0s

**Validation Accuracy**: 0.9036

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [7]:
from keras.layers import Dense, Activation, Flatten, Convolution2D
from keras.models import Sequential
from keras.utils import np_utils

data = GermanTrafficSignDataset()
data.configure(one_hot=True, train_validate_split_percentage=0)

X_train = data.train_orig
y_train = data.train_labels

X_train = data.normalize_data(X_train)
X_train = X_train.astype('float32')

# convert class vectors to binary class matrices
# Y_train = np_utils.to_categorical(y_train, nb_classes)

# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
model = Sequential(name='input')
model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(32, 32, 3)))
model.add(Dense(128, name='hidden1'))
model.add(Activation('relu'))
model.add(Dense(data.num_classes))
model.add(Activation('softmax', name='output'))

model.summary()

# TODO: Compile and train the model.
batch_size = 128
nb_classes = data.num_classes
nb_epoch = 2

model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

# history = model.fit(X_train, y_train,
#                    batch_size=batch_size, nb_epoch=nb_epoch,
#                     verbose=1)

from sklearn.model_selection import train_test_split

# TODO: Split some of the training data into a validation dataset.
X_train, X_val, y_train, y_val = train_test_split(
            X_train,
            y_train,
            test_size=0.25,
            random_state=832224)

# TODO: Compile and train the model to measure validation accuracy.
history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))

score = model.evaluate(X_val, y_val, verbose=1)

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 28800)         0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
hidden1 (Dense)                  (None, 128)           3686528     flatten_1[0][0]                  
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 128)           0           hidden1[0][0]                    
___________________________________________________________________________________________

**Validation Accuracy**: 0.9471

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [23]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
# TODO: Compile and train the model.

from keras.layers import Dense, Activation, Flatten, Convolution2D, MaxPooling2D
from keras.models import Sequential
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

data = GermanTrafficSignDataset()
data.configure(one_hot=True, train_validate_split_percentage=0)

X_train = data.train_orig
y_train = data.train_labels

X_train = data.normalize_data(X_train)
X_train = X_train.astype('float32')

# convert class vectors to binary class matrices
# Y_train = np_utils.to_categorical(y_train, nb_classes)

# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
model = Sequential(name='input')
model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, name='hidden1'))
model.add(Activation('relu'))
model.add(Dense(data.num_classes))
model.add(Activation('softmax', name='output'))

model.summary()

# TODO: Compile and train the model.
batch_size = 128
nb_classes = data.num_classes
nb_epoch = 2

model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

# TODO: Split some of the training data into a validation dataset.
X_train, X_val, y_train, y_val = train_test_split(
            X_train,
            y_train,
            test_size=0.25,
            random_state=832224)

# TODO: Compile and train the model to measure validation accuracy.
history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))

# score = model.evaluate(X_val, y_val, verbose=1)
# print('Validation (loss, accuracy): (%.3f, %.3f)' % (score[0], score[1]))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_16 (Convolution2D) (None, 30, 30, 32)    896         convolution2d_input_16[0][0]     
____________________________________________________________________________________________________
maxpooling2d_16 (MaxPooling2D)   (None, 15, 15, 32)    0           convolution2d_16[0][0]           
____________________________________________________________________________________________________
flatten_15 (Flatten)             (None, 7200)          0           maxpooling2d_16[0][0]            
____________________________________________________________________________________________________
hidden1 (Dense)                  (None, 128)           921728      flatten_15[0][0]                 
___________________________________________________________________________________________

**Validation Accuracy**: 0.9409

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [24]:
# TODO: Re-construct the network and add dropout after the pooling layer.
# TODO: Compile and train the model.

from keras.layers import Dense, Activation, Flatten, Convolution2D, MaxPooling2D, Dropout
from keras.models import Sequential
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

data = GermanTrafficSignDataset()
data.configure(one_hot=True, train_validate_split_percentage=0)

X_train = data.train_orig
y_train = data.train_labels

X_train = data.normalize_data(X_train)
X_train = X_train.astype('float32')

# convert class vectors to binary class matrices
# Y_train = np_utils.to_categorical(y_train, nb_classes)

# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
model = Sequential(name='input')
model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, name='hidden1'))
model.add(Activation('relu'))
model.add(Dense(data.num_classes))
model.add(Activation('softmax', name='output'))

model.summary()

# TODO: Compile and train the model.
batch_size = 128
nb_classes = data.num_classes
nb_epoch = 2

model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

# TODO: Split some of the training data into a validation dataset.
X_train, X_val, y_train, y_val = train_test_split(
            X_train,
            y_train,
            test_size=0.25,
            random_state=832224)

# TODO: Compile and train the model to measure validation accuracy.
history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))

# score = model.evaluate(X_val, y_val, verbose=1)
# print('Validation (loss, accuracy): (%.3f, %.3f)' % (score[0], score[1]))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_17 (Convolution2D) (None, 30, 30, 32)    896         convolution2d_input_17[0][0]     
____________________________________________________________________________________________________
maxpooling2d_17 (MaxPooling2D)   (None, 15, 15, 32)    0           convolution2d_17[0][0]           
____________________________________________________________________________________________________
dropout_3 (Dropout)              (None, 15, 15, 32)    0           maxpooling2d_17[0][0]            
____________________________________________________________________________________________________
flatten_16 (Flatten)             (None, 7200)          0           dropout_3[0][0]                  
___________________________________________________________________________________________

**Validation Accuracy**: 0.9349

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [29]:
from keras.layers import Dense, Activation, Flatten, Convolution2D, MaxPooling2D, Dropout
from keras.models import Sequential
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras import backend as K

data = GermanTrafficSignDataset()
data.configure(one_hot=True, train_validate_split_percentage=0)

X_train = data.train_orig
y_train = data.train_labels

X_train = data.normalize_data(X_train)
X_train = X_train.astype('float32')

batch_size = 128
nb_classes = data.num_classes
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 32, 32

# number of convolutional filters to use
nb_filters = 32

# number of channels for our input image
nb_channels = 3

# convolution kernel size
kernel_size = (5, 5)

# size of pooling area for max pooling
pool_size = (2, 2)

# number of neurons for our hidden layer
hidden_layer_neurons = 128

# A float between 0 and 1. Fraction of the input units to drop.
dropout_p_1, dropout_p_2 = 0.5, 0.5

# If Theano backend, input_shape is different so let's take care of that first
if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], nb_channels, img_rows, img_cols)
    input_shape = (nb_channels, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, nb_channels)
    input_shape = (img_rows, img_cols, nb_channels)


# build the model
model = Sequential(name='input')
model.add(Convolution2D(16, kernel_size[0], kernel_size[1], border_mode='valid', input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(dropout_p_1))
model.add(Flatten())
model.add(Dense(hidden_layer_neurons, name='hidden1'))
model.add(Activation('relu'))
# model.add(Dropout(dropout_p_2))
model.add(Dense(data.num_classes))
model.add(Activation('softmax', name='output'))

# print information about the model itself
model.summary()

# Compile and train the model.
model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

# Split some of the training data into a validation dataset.
X_train, X_val, y_train, y_val = train_test_split(
            X_train,
            y_train,
            test_size=0.25,
            random_state=832224)

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_23 (Convolution2D) (None, 28, 28, 32)    2432        convolution2d_input_21[0][0]     
____________________________________________________________________________________________________
activation_28 (Activation)       (None, 28, 28, 32)    0           convolution2d_23[0][0]           
____________________________________________________________________________________________________
convolution2d_24 (Convolution2D) (None, 24, 24, 32)    25632       activation_28[0][0]              
____________________________________________________________________________________________________
activation_29 (Activation)       (None, 24, 24, 32)    0           convolution2d_24[0][0]           
___________________________________________________________________________________________

**Best Validation Accuracy:** 0.9921

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data.

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [30]:
# with open('./test.p', mode='rb') as f:
#     test = pickle.load(f)
    
X_test = data.test_orig   # test['features']
y_test = data.test_labels # test['labels']
X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = y_test
# Y_test = np_utils.to_categorical(y_test, 43)

loss, accuracy = model.evaluate(X_test, Y_test)

print('loss: ', loss)
print('accuracy: ', accuracy)
print()

12630/12630 [==============================] - 15s    
loss:  0.298437518168
accuracy:  0.942834520972



**Test Accuracy:** 0.9428

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.